# CORRECTION OF SELECTION BIAS

In [1]:
from model_class_cb import CatBoostXT_BAG
from model_class_lgbm import LightGBMXT_BAG

In [2]:
from model_utils import *

In [3]:
from other_utils import compare_distributions

In [4]:
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
import pickle
from scipy.special import expit
import optuna
import json

In [5]:
data_dir = r'E:\Project\Dissertation'

In [6]:
with open(rf'{data_dir}\models\dict_cols.json') as json_file:
    dict_cols = json.load(json_file)

In [7]:
# separate features, target
list_features = dict_cols['list_features']
# features that starts with EXT_
list_ext_features = dict_cols['list_ext_features']
# list ethical features
list_ethical_features = dict_cols['list_ethical_features']
# list categorical features
list_cat_features = dict_cols['list_cat_features']

## Inverted prospensity weighting (IPW)

## Acceptance prospensity model

In [8]:
df = pd.read_parquet(rf'{data_dir}\data\processed\master_data_simulated_approval_and_reject.parquet')

In [9]:
label_accept = 'flag_accepted'

In [10]:
df.groupby(['flag_train_val','flag_accepted'])['flag_accepted'].count()

flag_train_val  flag_accepted
train           0                196339
                1                 34294
valid           0                 65442
                1                 11436
Name: flag_accepted, dtype: int64

In [11]:
X_accept_train = df.loc[df['flag_train_val']=='train', list_features]
X_accept_valid = df.loc[df['flag_train_val']=='valid', list_features]

y_accept_train = df.loc[df['flag_train_val']=='train', label_accept]
y_accept_valid = df.loc[df['flag_train_val']=='valid', label_accept]

In [12]:
accept_model = base_model = CatBoostXT_BAG(
    num_bag_folds=1,
    num_bag_repeats=1,
    random_state=1,
    cat_features=list_cat_features
)

In [ ]:
accept_model.params = {
    'iterations': 1000,
    'learning_rate': 0.05,
    'depth': 5,
    'l2_leaf_reg': 50,
    'border_count': 128,
    'auto_class_weights': 'SqrtBalanced',
    'early_stopping_rounds': None,
    'od_type': 'IncToDec',
    'od_wait': 200,
    'od_pval': 0.01,
    'boosting_type': 'Ordered',
    'bootstrap_type': 'Bernoulli',
    'loss_function': 'Logloss', 
    'eval_metric': 'Logloss',
    'task_type':'GPU',
    'devices':'0',
}

: 

In [ ]:
accept_model.fit(X_accept_train, y_accept_train,eval_set=(X_accept_valid, y_accept_valid))